In [1]:
#!pip install fpdf

In [2]:
import requests
import json
import os
from fpdf import FPDF

# Ollama API configuration for chat
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "phi3"

# Directory to save generated content
OUTPUT_DIR = "book_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_toc(subject):
    """Generate a Table of Contents for the given subject using chat API."""
    prompt = f"""
    You are an expert assistant skilled at creating book outlines.
    Create a detailed Table of Contents for a book on the following subject:
    Subject: {subject}
    
    Structure the response as a valid JSON object with chapters and sections:
    {{
        "Table_of_Contents": [
            {{
                "Chapter 1": "Title of Chapter",
                "Sections": [
                    "Section 1.1: Title of Section",
                    "Section 1.2: Title of Section"
                ]
            }},
            {{
                "Chapter 2": "Title of Chapter",
                "Sections": [
                    "Section 2.1: Title of Section",
                    "Section 2.2: Title of Section"
                ]
            }}
        ]
    }}
    """
    
    # Chat API expects a list of messages with roles like "system", "user", and "assistant"
    messages = [
        {"role": "system", "content": "You are an expert book creator."},
        {"role": "user", "content": prompt}
    ]
    
    # Sending the request with a high timeout (e.g., 120 seconds)
    payload = {"model": MODEL_NAME, "messages": messages}
    try:
        response = requests.post(OLLAMA_URL, json=payload, timeout=120)
        
        # Check if response status is OK
        if response.status_code != 200:
            raise Exception(f"Error from Ollama: {response.text}")

        # Print raw response to debug
        print("Raw Response Content:")
        print(response.text)  # print raw content for inspection
        
        # Attempt to parse JSON from the response
        try:
            content = response.json().get("response", "")
            toc = json.loads(content)  # try parsing it as JSON
            return toc
        except json.JSONDecodeError as e:
            raise Exception(f"Failed to parse ToC JSON: {e}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {str(e)}")

# Example usage
subject = "Prompt Engineering"
toc = generate_toc(subject)
print("Table of Contents generated:", toc)


Raw Response Content:
{"model":"phi3","created_at":"2024-12-01T06:29:29.381436Z","message":{"role":"assistant","content":"```"},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:29.505862Z","message":{"role":"assistant","content":"json"},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:29.632501Z","message":{"role":"assistant","content":"\n"},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:29.759783Z","message":{"role":"assistant","content":"{"},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:29.897984Z","message":{"role":"assistant","content":"\n"},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:30.03306Z","message":{"role":"assistant","content":"   "},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:30.169927Z","message":{"role":"assistant","content":" \""},"done":false}
{"model":"phi3","created_at":"2024-12-01T06:29:30.305437Z","message":{"role":"assistant","content":"Table"},"done":false}
{"model":"phi3","cre

Exception: Failed to parse ToC JSON: Extra data: line 2 column 1 (char 120)